<a href="https://colab.research.google.com/github/rcz87/TELEGRAM_CRYPTO-BOT/blob/main/Salinan_dari_Salinan_dari_Salinan_dari_bot_pertamaku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot --quiet


In [ ]:
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters

In [ ]:
TELEGRAM_TOKEN = "7695838700:AAFoZve12b53RzL8pu_swAnFkjaqXz43zKU"

In [ ]:
from telegram.ext import Updater, CommandHandler


In [ ]:
!pip install "python-telegram-bot[job-queue]" --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
from telegram.ext import ContextTypes

In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes

In [ ]:
from telegram import Update

In [ ]:
!pip install apscheduler --quiet


In [ ]:
!pip install python-telegram-bot --upgrade --quiet
!pip install apscheduler --quiet


In [ ]:
# Daftar coin populer OKX untuk sementara (nanti bisa otomatis)
COIN_LIST = [
    "BTC-USDT", "ETH-USDT", "SOL-USDT", "XRP-USDT", "DOGE-USDT",
    "ADA-USDT", "AVAX-USDT", "DOT-USDT", "SHIB-USDT", "LINK-USDT",
    "TRX-USDT", "TON-USDT", "MATIC-USDT", "BCH-USDT", "LTC-USDT",
    "APT-USDT", "ARB-USDT", "OP-USDT", "INJ-USDT", "RNDR-USDT",
    # tambahkan lebih banyak jika mau
]

In [ ]:
async def simple_check(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        # Ambil pesan yang dimasukkan oleh user dan ubah ke uppercase
        text = update.message.text.strip().upper()  # Mengubah semua input menjadi huruf besar
        words = text.split()  # Pisahkan input menjadi list berdasarkan spasi

        # Map timeframe yang valid ke format yang diterima oleh OKX
        tf_map = {
            '3 MENIT': '3 Menit', '5 MENIT': '5 Menit', '30 MENIT': '30 Menit', '1 JAM': '1 Jam',
            '2 JAM': '2 Jam', '4 JAM': '4 Jam', '1 HARI': '1 Hari', '3 HARI': '3 Hari',
            '3M': '3 Menit', '5M': '5 Menit', '30M': '30 Menit', '1H': '1 Jam', '2H': '2 Jam',
            '4H': '4 Jam', '1D': '1 Hari', '3D': '3 Hari'
        }

        pairs = []
        unknowns = []

        # Parsing input
        i = 0
        while i < len(words):
            coin = words[i]
            tf = '5 Menit'  # Default timeframe

            # Debug: Cek input yang diterima
            print(f"Memeriksa input: {coin} dengan timeframe {tf}")

            # Menangani jika hanya angka '5', anggap itu '5 Menit'
            if coin.isdigit() and int(coin) == 5:
                tf = '5 Menit'  # Ubah menjadi '5 Menit' jika hanya '5' yang dimasukkan
            elif i + 1 < len(words) and words[i + 1] in tf_map:
                tf = tf_map[words[i + 1]]  # Menyesuaikan timeframe sesuai input
                i += 1  # Skip next word which is the timeframe

            # Memastikan coin diubah menjadi pair -USDT jika belum ada
            if not coin.endswith("-USDT"):
                coin = coin + "-USDT"

            # Debug: Cek apakah coin ada di COIN_LIST
            print(f"Memeriksa coin: {coin} ada di COIN_LIST: {coin in COIN_LIST}")

            # Periksa apakah coin ada di COIN_LIST
            if coin in COIN_LIST:
                pairs.append((coin, tf))  # Coin valid
            else:
                unknowns.append(coin)  # Coin tidak valid
            i += 1

        # Jika ada coin invalid, kirimkan pesan
        if unknowns:
            await update.message.reply_text("⚠️ Coin tidak valid: " + ', '.join(unknowns))

        if not pairs:
            await update.message.reply_text("⚠️ Tidak ada coin valid yang kamu masukkan.")
            return

        # Proses coin yang valid
        for coin, tf in pairs:
            inst = coin
            try:
                # Coba ambil data orderbook dari OKX
                ob_url = f"https://www.okx.com/api/v5/market/books?instId={inst}&sz=5"
                ob_res = requests.get(ob_url).json()
                orderbook = ob_res['data'][0] if ob_res['code'] == '0' else None

                # Debug: Cek apakah orderbook berhasil diambil
                print(f"Orderbook untuk {coin}: {orderbook}")

            except Exception as e:
                orderbook = None
                print(f"Error fetching orderbook for {coin}: {e}")

            # Ambil data candle dan lakukan analisis
            df = get_candles(inst, bar=tf)

            # Debug: Cek data candle yang diambil
            print(f"Data candle untuk {coin} ({tf}): {df.head()}")

            df = calculate_indicators(df)
            signals = analyze(df, orderbook=orderbook)
            latest_price = df['close'].iloc[-1]

            # Membuat output dengan format yang rapi
            output_text = f"""━━━━━━━━━━━━━━━━━━━━━━━
📊 *ANALISA TEKNIKAL*
━━━━━━━━━━━━━━━━━━━━━━━

💎 *{inst}* — {tf}
💰 Harga: {latest_price:.2f}

{chr(10).join(signals)}
━━━━━━━━━━━━━━━━━━━━━━━"""

            await update.message.reply_text(output_text, parse_mode='Markdown')

    except Exception as e:
        await update.message.reply_text(f"❌ Error: {str(e)}")


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters

# Inisialisasi bot
application = ApplicationBuilder().token("7695838700:AAFoZve12b53RzL8pu_swAnFkjaqXz43zKU").build()

# Handler start
async def start(update, context):
    await update.message.reply_text("Halo! Kirimkan coin dan timeframe seperti: SOL 5 Menit")

application.add_handler(CommandHandler("start", start))
application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, simple_check))

# Jalankan bot dengan pengecekan agar tidak crash jika sudah jalan
async def main():
    try:
        print("Menjalankan bot...")
        await application.initialize()
        await application.start()
        print("Bot aktif dan menunggu pesan...")
        await application.run-polling()
    except RuntimeError as e:
        if "already running" in str(e):
            print("Bot sudah berjalan sebelumnya.")
        else:
            raise e
import nest_asyncio
nest_asyncio.apply()

await main()